In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c0e77e90",
   "metadata": {},
   "outputs": [],
   "source": [
    "import os\n",
    "import gradio as gr\n",
    "from langchain_groq import ChatGroq\n",
    "from langchain_community.chat_message_histories import ChatMessageHistory\n",
    "from langchain_core.chat_history import BaseChatMessageHistory\n",
    "from langchain_core.runnables.history import RunnableWithMessageHistory\n",
    "from langchain_core.prompts import ChatPromptTemplate\n",
    "from langchain.prompts import MessagesPlaceholder\n",
    "from operator import itemgetter\n",
    "from langchain_core.runnables import RunnablePassthrough\n",
    "from langchain_core.messages import trim_messages, HumanMessage\n",
    "from dotenv import load_dotenv\n",
    "load_dotenv()\n",
    "llm = ChatGroq(model=\"llama-3.1-8b-instant\",api_key='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')\n",
    "store = {}\n",
    "def get_session_history(session_id: str) -> BaseChatMessageHistory:\n",
    "    if session_id not in store:\n",
    "        store[session_id] = ChatMessageHistory()\n",
    "    return store[session_id]\n",
    "prompt = ChatPromptTemplate.from_messages([\n",
    "    (\"system\", \"You are a knowledgeable and authoritative guide on organ donation and medical topics. \"\n",
    "               \"Your role is to provide accurate, reliable guidance on organ donation procedures, \"\n",
    "               \"medical conditions, and related healthcare matters. You must strictly refuse to answer any \"\n",
    "               \"question that is not related to organ donation or medical topics. If a question is unrelated, \"\n",
    "               \"respond with: 'This question is outside the scope of Donix. I cannot answer it.' \"\n",
    "               \"Remember past conversations to provide context-aware responses in {language}.\"),\n",
    "    MessagesPlaceholder(variable_name=\"messages\")\n",
    "])\n",
    "trimmer = trim_messages(\n",
    "    max_tokens=900,\n",
    "    strategy=\"last\",\n",
    "    token_counter=llm,\n",
    "    include_system=True,\n",
    "    allow_partial=False,\n",
    "    start_on=\"human\",\n",
    ")\n",
    "chain = (\n",
    "    RunnablePassthrough.assign(messages=itemgetter(\"messages\") | trimmer)\n",
    "    | prompt\n",
    "    | llm\n",
    ")\n",
    "config = {\"configurable\": {\"session_id\": \"default_session\"}}\n",
    "with_message_history = RunnableWithMessageHistory(\n",
    "    chain,\n",
    "    get_session_history,\n",
    "    input_messages_key=\"messages\",\n",
    ")\n",
    "def chat_interface(message, language=\"English\", session_id=\"default_session\"):\n",
    "    response = with_message_history.invoke(\n",
    "        {\n",
    "            \"messages\": HumanMessage(message),\n",
    "            \"language\": language\n",
    "        },\n",
    "        config={\"configurable\": {\"session_id\": session_id}}\n",
    "    ).content\n",
    "    return response\n",
    "iface = gr.Interface(\n",
    "    fn=chat_interface,\n",
    "    inputs=[\n",
    "        gr.Textbox(label=\"Message\"),\n",
    "        gr.Dropdown([\"English\", \"Hindi\", \"Sanskrit\"], label=\"Language\", value=\"English\"),\n",
    "        gr.Textbox(label=\"Session ID\", value=\"default_session\")\n",
    "    ],\n",
    "    outputs=\"text\",\n",
    "    title=\"Donix Organ Donation Chatbot\",\n",
    "    description=\"A chatbot specialized in organ donation and medical guidance.\"\n",
    ")\n",
    "if __name__ == \"__main__\":\n",
    "    iface.launch()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d2ce214d",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "49d5469c",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "9ef61129",
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "abed45a6",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.12.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}